##### build freesurfer and set up environmental variables

In [5]:
!source /projects/shibo/Visual-Feature-Attribution-Using-Wasserstein-GANs-Pytorch/ADNI_preprocessing/build_freesurfer.sh

freesurfer bin already exists
Reading package lists... Done
Building dependency tree       
Reading state information... Done
bc is already the newest version (1.07.1-2).
unzip is already the newest version (6.0-21ubuntu1).
libjpeg62-dev is already the newest version (1:6b2-3).
tcsh is already the newest version (6.20.00-7).
binutils is already the newest version (2.30-21ubuntu1~18.04.3).
libgomp1 is already the newest version (8.4.0-1ubuntu1~18.04).
perl is already the newest version (5.26.1-6ubuntu0.3).
psmisc is already the newest version (23.1-1ubuntu0.1).
tar is already the newest version (1.29b-2ubuntu0.1).
uuid-dev is already the newest version (2.31.1-0.4ubuntu3.6).
vim-common is already the newest version (2:8.0.1453-1ubuntu1.3).
The following packages were automatically installed and are no longer required:
  cdbs debhelper dh-autoreconf dh-strip-nondeterminism dh-translations
  gdal-data ibverbs-providers intltool jq libaec0 libarchive-cpio-perl
  libarchive-zip-perl libarma

##### begin autorecon1 steps (first 5 steps in recon-all)
##### remove all previous test results

In [ ]:
rm -rf /usr/local/freesurfer/subjects/recon_test/ 

##### execute autorecon1 (5 steps)
###### -motioncor: correct for small motions volumes and then average them together
    inputs: orig.mgz  outputs: mri/orig.mgz 
###### -talairach: computes the affine transform from the orig volume to the MNI305 atlas using the MINC program mritotal
    inputs: orig.mgz  outputs: transforms/talairach.xfm transforms/talairach_avi.log  
###### -nuintensitycor:  corrects for intensity non-uniformity in MR data, making relatively few assumptions about the data
    inputs: orig.mgz talairach.xfm  outputs:  mri/nu.mgz
###### -normalization: performs intensity normalization of the orig volume and places the result in mri/T1.mgz
    inputs: nu.mgz  outputs: mri/T1.mgz
###### -skullstrip: removes the skull from mri/T1.mgz and stores the result in mri/brainmask.auto.mgz and mri/brainmask.mgz
    inputs: nu.mgz  outputs: mri/brainmask.mgz

In [ ]:
recon-all -i /projects/shibo/Visual-Feature-Attribution-Using-Wasserstein-GANs-Pytorch/ADNI_preprocessing/test_images/002_S_1018/ADNI_002_S_1018_MR_MPRAGE_br_raw_20061219201951192_1_S24311_I33831.dcm -subject recon_test -autorecon1    

##### now begin fetching (clearing all fetched results)

In [ ]:
!rm -rf fetch_preprocessed/
!ls
!mkdir fetch_preprocessed

##### fetch motioncor

In [ ]:
!mkdir fetch_preprocessed/motioncor
!cp /usr/local/freesurfer/subjects/recon_test/mri/orig.mgz fetch_preprocessed/motioncor

##### fetch nuintensitycor

In [ ]:
!mkdir fetch_preprocessed/nuintensitycor
!cp /usr/local/freesurfer/subjects/recon_test/mri/nu.mgz fetch_preprocessed/nuintensitycor
!ls fetch_preprocessed/nuintensitycor/

##### fetch talairach

In [ ]:
!mkdir fetch_preprocessed/talairach
!cp /usr/local/freesurfer/subjects/recon_test/mri/transforms/talairach_with_skull.lta fetch_preprocessed/talairach

##### fetch normalization

In [ ]:
!mkdir fetch_preprocessed/normalization
!cp /usr/local/freesurfer/subjects/recon_test/mri/T1.mgz fetch_preprocessed/normalization

##### fetch skullstrip

In [ ]:
!mkdir fetch_preprocessed/skullstrip
!cp /usr/local/freesurfer/subjects/recon_test/mri/brainmask.mgz fetch_preprocessed/skullstrip

##### perform co-registration with mni-space

In [1]:
# using freesurfer
# !mkdir fetch_preprocessed/mni_registered
# !mri_robust_register --mov fetch_preprocessed/skullstrip/brainmask.mgz --dst test_images/mni_space/mni_icbm152_t1_tal_nlin_asym_09c_mask.nii --lta fetch_preprocessed/mni_registered/reg.lta --satit 
# !mri_convert fetch_preprocessed/skullstrip/brainmask.mgz fetch_preprocessed/mni_registered/mni_reg.nii --apply_transform fetch_preprocessed/mni_registered/reg.lta

# using spm
!sh mgz2nii_fetched.sh
!python3 spm_coregister.py

mgz2nii_fetched.sh: line 2: mri_convert: command not found
mgz2nii_fetched.sh: line 3: mri_convert: command not found
mgz2nii_fetched.sh: line 4: mri_convert: command not found
mgz2nii_fetched.sh: line 5: mri_convert: command not found
200526-10:18:32,548 nipype.utils INFO:
	 No new version available.


##### .mgz to .nii conversion in all 5 folders


In [ ]:
!sh mgz2nii_fetched.sh
!ls fetch_preprocessed/motioncor fetch_preprocessed/nuintensitycor fetch_preprocessed/normalization fetch_preprocessed/talairach/ fetch_preprocessed/skullstrip